In [ ]:
import pandas as pd
import re
import sqlalchemy
import unicodedata
import difflib
import xlwings as xw
from datetime import datetime
from googletrans import Translator
from os import listdir
from os.path import isfile, join

In [ ]:
def get_full_cost(x, full_cost_columns):
    cost = x[['Brutto', 'ZUS pracodawcy', 'ZUS Pracodawca', 'Skladki FGSP', 'Składki FGŚP', 'Skladki FP']].sum()
    cost = cost - x[[c for c in full_cost_columns if 'Zas' in c]].sum()
    
    return cost

def get_protected_sheet_data(PATH):
    wb = xw.Book(PATH)
    
    salaries = []
    for sheet in wb.sheets:
        print(sheet)
        
        data_from_sheet = sheet.range('A1').expand().options(pd.DataFrame, index=False, header=True).value
        
        data_from_sheet['Etat'] = data_from_sheet.apply(lambda x: x['Wymiar etatu L']/x['Wymiar etatu M'], axis=1)
        
        data_from_sheet.rename(columns={"Tytul": "Stanowisko", 
                                        "Kwota": "Stawka",
                                        "kwota": "Stawka",
                                        "Wartość": "Stawka",
                                        "wartość": "Stawka",
                                       "ETAT": 'Etat'}, inplace=True)    
            
        data_from_sheet['FinanceSheet'] = sheet.name

        data_from_sheet['Date'] = data_from_sheet['Data']
        full_cost_columns = data_from_sheet.columns.values
        data_from_sheet['EnovaEmployerCost'] = data_from_sheet.apply(lambda x: get_full_cost(x, full_cost_columns), axis=1)
        
        salaries.append(data_from_sheet[['Kod', 'Pracownik', 'Stanowisko', 'Etat',
                                         'Typ umowy', 'FinanceSheet', 'Date', 'Stawka', 'EnovaEmployerCost']])
    
    return salaries

def etat_to_number(x):
    g = re.match('([0-9])/([0-9])', str(x).replace("'", ""))
    if g is None:
        return g
    g = g.groups()
    if len(g) == 1:
        return g[0]
    return float(g[0])/float(g[1])

def get_name(x):
    x = unicodedata.normalize('NFD', x.lower()).replace('ł', 'l').encode('ascii', 'ignore').decode()
    if '(' in x:
        full_name = re.match('(.{1,}) (.{1,}) \(.{1,}\)', x).groups()
    else:
        full_name = re.match('(.{1,}) (.{1,})', x).groups()
    return full_name[1].title() + ' ' + full_name[0].title()

In [ ]:
mypath = 'C:\GIT\Salary\data\ContractValue'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
onlyfiles = [file for file in onlyfiles if not '~' in file]
onlyfiles

In [ ]:
salaries_array = []
for name in onlyfiles:
    print(name)
    salaries_array.append(pd.concat(get_protected_sheet_data('data\ContractValue\\' + name)))

In [ ]:
salaries = pd.concat(salaries_array)
    
salaries = salaries.rename(columns={
    'Kod': 'FinanceCode', 
    'Pracownik': 'FinanceName', 
    'Stanowisko': 'FinancePosition', 
    'Etat': 'PartTime', 
    'Stawka': 'ContractValue',
    'Typ umowy': 'ContractType',
})

#salaries['PartTime'] = salaries['PartTime'].apply(etat_to_number) 
salaries = salaries.drop_duplicates()

salaries['CorrectedFinanceName'] = salaries['FinanceName'].apply(get_name)

In [ ]:
salaries.head()

In [ ]:
salaries['Date'] = salaries['Date'].apply(lambda x: x.replace(day=1))
salaries.groupby(['Date']).count()

In [ ]:
salaries = salaries[~salaries['FinancePosition'].isnull()]
columns = ['FinanceCode', 'FinanceName', 'FinancePosition', 'PartTime', 'ContractType', 'FinanceSheet', 'Date', 'CorrectedFinanceName']

In [ ]:
salaries = salaries.groupby(columns).sum().reset_index()

In [ ]:
salaries = salaries[~(salaries['ContractValue'].isnull() & salaries['EmployerCost'].isnull())]
salaries.head()

In [ ]:
salaries.head()
salaries['PartTime'] = salaries['PartTime'].fillna(1)
salaries.shape

In [ ]:
salaries['Date'] = salaries['Date'].apply(lambda x: x.date())

In [ ]:
DM_SERVER = "OBJPLDMSQL"
DM_DATABASE = "Structure"
DM_SCHEMA = "dbo"
DM_DB_CONNECTION = 'mssql+pyodbc://@' + DM_SERVER + '/' + DM_DATABASE + '?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes'
DM_ENGINE = sqlalchemy.create_engine(DM_DB_CONNECTION, pool_size=20, max_overflow=100)

In [ ]:
employees = pd.read_sql('''
select * FROM [Structure].[dbo].[Structure_Monthly] where Date >= '{0} 00:00:00.000' and Date <= '{1} 00:00:00.000'
'''.format(salaries['Date'].min(), salaries['Date'].max()), DM_ENGINE.connect())

In [ ]:
employees['Level'] = employees['Level'].replace({'': 'Regular'})

In [ ]:
names_changes = pd.read_sql('''
SELECT distinct concat([First_Name], ' ', [Surname]) as NameFrom
      ,concat([First_Name], ' ', [Previous_Surname]) as NameTo
FROM [Translations].[dbo].[Maiden_Name]

UNION ALL

SELECT distinct concat([First_Name], ' ', [Previous_Surname]) as NameFrom
      ,concat([First_Name], ' ', [Surname]) as NameTo
FROM [Translations].[dbo].[Maiden_Name]
''', DM_ENGINE.connect())

In [ ]:
translator = Translator()
positions = salaries['FinancePosition'].unique()
translations = []
for x in positions:
    #print(x)
    try:
        translations.append(translator.translate(x.replace('/', ''), dest='en').text)
    except Exception as e:
        translations.append(x)
positions_translations = pd.DataFrame({'FinancePosition': positions, 'FinanceEnglishPosition': translations})
positions_translations.head()

In [ ]:
salaries = salaries.merge(positions_translations, how='left')
salaries.shape

In [ ]:
def get_matched_record(record, name_column='CorrectedFinanceName', position_column='FinanceEnglishPosition', prefix=''):
    record = record.iloc[0]
    x = record[name_column]
    #print(x)
    
    ratios = pd.DataFrame({'Name': employees['Name'].unique(),
                  'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in employees['Name'].unique()]
                 })
    
    max_ratio = ratios['Ratio'].max() - 0.1
    if max_ratio < 0.85:
        print(x)
        print(ratios['Ratio'].max())
        
        ratios_maiden = pd.DataFrame({'Name': names_changes['NameFrom'].tolist(),
              'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in names_changes['NameFrom'].tolist()]
             })
        max_ratio_maiden = ratios_maiden['Ratio'].max()
        matched_name_maiden = ratios_maiden[ratios_maiden['Ratio'] >= max_ratio_maiden]
        if matched_name_maiden.shape[0] >= 1 and (max_ratio_maiden - 0.1) > max_ratio:
            x = names_changes['NameTo'][names_changes['NameFrom'] == matched_name_maiden['Name'].iloc[0]].iloc[0]
            
            ratios = pd.DataFrame({'Name': employees['Name'].unique(),
                  'Ratio': [difflib.SequenceMatcher(None, x, name).ratio() for name in employees['Name'].unique()]
                 })
    
            max_ratio = ratios['Ratio'].max() - 0.1
    
    matched_name = ratios[ratios['Ratio'] >= max_ratio]

    x = record[position_column]
    #TODO
    #x= translator.translate(x, dest='en').text

    positions = employees[['Name', 'Position']].drop_duplicates().merge(matched_name, how='inner', on='Name')[['Name', 'Position', 'Ratio']]

    pos_ratios = pd.DataFrame({
        prefix + 'MatchedName': positions['Name'].tolist(),
        prefix + 'MatchedNameRatio': positions['Ratio'].tolist(),
        prefix + 'MatchedPosition': positions['Position'].tolist(),
        prefix + 'MatchedPositionRatio': [difflib.SequenceMatcher(None, x.lower() if x is not None else ' '
                                                         , name.lower() if name is not None else ' ').ratio() for name in positions['Position'].tolist()]
    })
    pos_max_ratio = pos_ratios[prefix + 'MatchedPositionRatio'].max()
    matched_pos = pos_ratios[pos_ratios[prefix + 'MatchedPositionRatio'] >= pos_max_ratio]
    
    matched_pos = matched_pos[matched_pos[prefix + 'MatchedNameRatio'].max() == matched_pos[prefix + 'MatchedNameRatio']]

    return matched_pos

In [ ]:
matched_employees = salaries.groupby(['CorrectedFinanceName', 'FinanceEnglishPosition']).apply(get_matched_record).reset_index()

In [ ]:
matched_employees = matched_employees.drop(['level_2'], axis=1)

In [ ]:
merged_salary = salaries.merge(matched_employees, how="outer")
merged_salary.shape

In [ ]:
merged_salary['Ratio'] = merged_salary['MatchedNameRatio'] * merged_salary['MatchedPositionRatio']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

merged_salary['Ratio'][~merged_salary['MatchedPosition'].isnull()].hist()

In [ ]:
merged_salary[(~merged_salary['MatchedPosition'].isnull()) & (merged_salary['Ratio'] < 0.2)][['FinanceName', 'FinancePosition', 'MatchedPosition']].drop_duplicates()

In [ ]:
employees = employees[['Name', 'Position', 'Tribe', 'Subtribe', 'Guild', 'Subguild', 'Office', 'Level', 'values',
                      'LeaveDate', 'EmploymentDate', 'Management', 'Date', 'Backoffice', 'LastWorkingDate', 'Sex']]

In [ ]:
employees['Date'] = employees['Date'].apply(lambda x : x.date())

In [ ]:
merged_salary['Date'] = pd.to_datetime(merged_salary['Date'])

In [ ]:
merged_salary['Date'] = merged_salary['Date'].apply(lambda x : x.date())

In [ ]:
merged_salary = merged_salary.merge(employees, how="outer", left_on=['MatchedName', 'Date'], right_on=['Name', 'Date'])

In [ ]:
merged_salary.head()
salaries.shape[0], merged_salary.shape[0]

In [ ]:
merged_salary['Date'].unique()

In [ ]:
count = merged_salary.groupby(['Name', 'Date']).apply(lambda x: pd.DataFrame({'Count': [x.shape[0]]}))
count[count['Count'] > 1]

In [ ]:
merged_salary.head()

In [ ]:
merged_salary[(merged_salary['Name'] == 'Monika Grzech')]['Ratio'].head()

In [ ]:
merged_salary['ContractType'].unique()

In [ ]:
FIN_SERVER = "OBJPLDMFIN1"
FIN_DATABASE = "Cost"
FIN_SCHEMA = "dbo"
FIN_DB_CONNECTION = 'mssql+pyodbc://@' + FIN_SERVER + '/' + FIN_DATABASE + '?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes'
FIN_ENGINE = sqlalchemy.create_engine(FIN_DB_CONNECTION, pool_size=20, max_overflow=100)

In [ ]:
contract_data = merged_salary

In [ ]:
#merged_salary = contract_data

In [ ]:
merged_salary.to_sql('Salaries_ContractValues', FIN_ENGINE.connect(), if_exists='replace', index=False)

In [ ]:
merged_salary = contract_data

In [ ]:
merged_salary.head()

In [ ]:
mypath = 'C:\GIT\Salary\data\EmployerCost\Hansa'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = [file for file in onlyfiles if not '~' in file and 'B2B' in file]
onlyfiles

In [ ]:
hansa_array = []
for file in onlyfiles:
    print(file)
    hansa_data = pd.read_excel('data\EmployerCost\Hansa\\' + file, 'Hansa Report')
    hansa_data['HansaSheet'] = file[:8]
    hansa_data['Opis'] = hansa_data['Unnamed: 1']
    hansa_data['Date'] = datetime.strptime(file[9:16], '%m.%Y').date()
    hansa_array.append(hansa_data)

hansa_array[1].columns

In [ ]:
mypath = 'C:\GIT\Salary\data\EmployerCost\Hansa'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = [file for file in onlyfiles if not '~' in file and 'B2B' not in file]
onlyfiles

In [ ]:
for file in onlyfiles:
    for t in ['OBSS', 'OITS']:
        print(file)
        print(t)
        hansa_data = pd.read_excel('data\EmployerCost\Hansa\\' + file, sheetname=t, skiprows=1)
        print(hansa_data.shape)
        hansa_data['HansaSheet'] = t
        hansa_data['Konto'] = hansa_data['Obiekt']
        hansa_data['Date'] = datetime.strptime(file[:7], '%m_%Y').date()
        hansa_array.append(hansa_data)

In [ ]:
hansa = pd.concat(hansa_array)

In [ ]:
[ha.shape[0] for ha in hansa_array]

In [ ]:
hansa.head()

In [ ]:
hansa = pd.DataFrame({
    'HansaCode': hansa['Konto'],
    'HansaName': hansa['Opis'].apply(lambda x: unicodedata.normalize('NFD', str(x).strip()).replace('ł', 'l').replace('Ł', 'L').encode('ascii', 'ignore').decode()),
    'HansaEmployerCost': hansa['Okres Wn'],
    'HansaSheet': hansa['HansaSheet'],
    'Date': hansa['Date']
})

In [ ]:
hansa.groupby(['Date']).count()

In [ ]:
def str_to_float(x):
    if re.match('[0-9]{1,}.[0-9]{1,}.[0-9]{1,},[0-9]{1,}', str(x)) is not None:
        return float(str(x).replace('.', '').replace(',', '.'))
    elif re.match('-{0,}[0-9]{1,}.[0-9]{1,},[0-9]{1,}', str(x)) is not None:
        return float(str(x).replace('.', '').replace(',', '.'))
    return float(str(x))

hansa['HansaEmployerCost'] = hansa['HansaEmployerCost'].apply(str_to_float)

In [ ]:
hansa = hansa[~hansa['HansaName'].isin(['Miscellaneous', 'Wynagrodzenia - agencje pracy', 'wolne']) & ~hansa['HansaEmployerCost'].isnull()]

In [ ]:
hansa['HansaName'] = hansa['HansaName'].replace({'Pawel Ozog QE': 'Pawel Ozog', 'Daniel Wiczak': 'Daniel Wilczak'})

In [ ]:
hansa.head()

In [ ]:
hansa = hansa.groupby(['Date', 'HansaCode', 'HansaName', 'HansaSheet']).sum().reset_index()

In [ ]:
merged_hansa = hansa.merge(merged_salary, how='outer', left_on=['HansaName', 'Date'], right_on=['Name', 'Date'])
merged_hansa.shape, merged_salary.shape

In [ ]:
merged_hansa[(merged_hansa['Name'].isnull()) & (~merged_hansa['HansaName'].isnull())]

In [ ]:
mypath = 'C:\GIT\Salary\data\EmployerCost\Enova'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = [file for file in onlyfiles if not '~' in file and 'LISTA' in file]
onlyfiles

In [ ]:
full_cost = []
for PATH in onlyfiles:
    wb = xw.Book('data\EmployerCost\Enova\\' +  PATH)
    
    for sheet in wb.sheets:
        data_from_sheet = sheet.range('A1').expand().options(pd.DataFrame, index=False, header=True).value

        data_from_sheet['EnovaName'] =  data_from_sheet['NAZWISKO IMIĘ']
        data_from_sheet['EnovaPosition'] =  data_from_sheet['SATANOWISKO']

        data_from_sheet['Date'] = data_from_sheet['Data'] #datetime(int(date_str[1]), int(date_str[0]), 1)
        data_from_sheet['Path'] = PATH

        full_cost.append(data_from_sheet)

In [ ]:
full_cost = pd.concat(full_cost)
full_cost = full_cost[~full_cost['EnovaName'].isnull()]

In [ ]:
def get_full_cost(x):
    cost = x[['Brutto', 'ZUS pracodawcy', 'ZUS Pracodawca', 'Skladki FGSP', 'Składki FGŚP', 'Skladki FP']].sum()
    cost = cost - x[[c for c in full_cost_columns if 'Zas' in c]].sum()
    
    return cost

def get_zasilek(x):
    return x[[c for c in full_cost_columns if 'Zas' in c]].sum()

In [ ]:
full_cost_columns = full_cost.columns.values
full_cost['EnovaEmployerCost'] = full_cost.apply(get_full_cost, axis=1)
full_cost['EnovaDole'] = full_cost.apply(get_zasilek, axis=1)

In [ ]:
full_cost['EnovaDole'].hist()

In [ ]:
full_cost['Date'] = full_cost['Date'].apply(lambda x: datetime.strptime(x, '%m.%Y') if '.' in str(x) else x)

In [ ]:
full_cost['Date'] = full_cost['Date'].apply(lambda x: x.replace(day=1).date())

In [ ]:
full_cost = full_cost[['Date', 'EnovaName', 'EnovaEmployerCost', 'EnovaPosition', 'EnovaDole']]

In [ ]:
full_cost[full_cost['EnovaName'] == 'SZATANIK BARTOSZ']

In [ ]:
full_cost = full_cost.groupby(['Date', 'EnovaName', 'EnovaPosition']).sum().reset_index()

In [ ]:
full_cost.head()

In [ ]:
full_cost = full_cost.drop_duplicates()

In [ ]:
full_cost.head()

In [ ]:
# Translations
positions = full_cost['EnovaPosition'].unique()
translations = []
for x in positions:
    #print(x)
    try:
        translations.append(translator.translate(x.replace('/', ''), dest='en').text)
    except Exception as e:
        translations.append(x)
positions_translations = pd.DataFrame({'EnovaPosition': positions, 'EnovaEnglishPosition': translations})
positions_translations.head()

In [ ]:
full_cost = full_cost.merge(positions_translations, how='left')

In [ ]:
full_cost = full_cost[['Date', 'EnovaName', 'EnovaEmployerCost', 'EnovaDole', 'EnovaEnglishPosition', 'EnovaPosition']]

In [ ]:
full_cost['CorrectedEnovaName'] = full_cost['EnovaName'].apply(get_name)

In [ ]:
# Names
matched_employees = full_cost.groupby(['CorrectedEnovaName', 'EnovaEnglishPosition']).apply(lambda x: get_matched_record(x, 'CorrectedEnovaName', 'EnovaEnglishPosition', 'Enova')).reset_index()

In [ ]:
matched_employees = matched_employees.drop('level_2', axis=1)

In [ ]:
full_cost_names = full_cost.merge(matched_employees, how="outer")

In [ ]:
full_cost_names[full_cost_names['EnovaName'] == 'ROMANOWSKI TOMASZ']

In [ ]:
count = full_cost_names.groupby(['Date', 'EnovaName', 'EnovaPosition']).apply(lambda x: pd.DataFrame({'Count': [x.shape[0]]})).reset_index()
count[count['Count'] > 1]

In [ ]:
merged_all = merged_hansa.merge(full_cost_names, how='outer', left_on=['Name', 'Date'], right_on=['EnovaMatchedName', 'Date'])

In [ ]:
merged_all = merged_hansa

In [ ]:
merged_all['EmployerCost'] = merged_all['EnovaEmployerCost'].fillna(0) + merged_all['HansaEmployerCost'].fillna(0)

In [ ]:
merged_all.columns

In [ ]:
merged_all['PartTime'] = merged_all['PartTime'].replace({0.0: 1.0})
merged_all['FullEmployerCost'] = merged_all['EmployerCost']/merged_all['PartTime']

In [ ]:
merged_all['EmployerToContractRatio'] = merged_all['EmployerCost']/merged_all['ContractValue']

In [ ]:
merged_all['IsDirector'] = merged_all['Position'].isin(['Group Finance Director', 'Director', 'Managing Director', 'Marketing Director'])

In [ ]:
merged_all['Computation_Time'] = datetime.now()

In [ ]:
merged_all = merged_all.drop_duplicates()
merged_all['ID'] = list(range(merged_all.shape[0]))

In [ ]:
maternity = pd.read_sql('''
SELECT distinct Name, Date
FROM [AvailableDays].[dbo].[TimesheetDetailed]
where [IsMaternity] = 1''', DM_ENGINE.connect())
maternity.head()

In [ ]:
maternity['IsMaternity'] = 1

In [ ]:
maternity['Date'] = maternity['Date'].apply(lambda x: x.date())

In [ ]:
merged_all['DateBeforeEmployment'] = merged_all['Date'] < merged_all['EmploymentDate'].apply(lambda x: datetime.strptime(str(x)[:10], '%Y-%m-%d').date() if str(x) != 'nan' and x is not None else None)

In [ ]:
merged_all['FullTimeContractValue'] = merged_all['ContractValue']/merged_all['PartTime']

In [ ]:
merged_all = merged_all.sort_values(['Date'])
#merged_all = merged_all.drop(['PreviousContractValue', 'Raise'], axis=1)
def get_previous(x):
    return pd.DataFrame({'PreviousContractValue': x['ContractValue'].shift(1),
                        'Raise': x['ContractValue'] - x['ContractValue'].shift(1),
                        'Date': x['Date'],
                        'Name': x['Name']})
previous = merged_all.groupby(['Name']).apply(get_previous)
merged_all = merged_all.merge(previous, how='left')

In [ ]:
merged_all.merge(maternity, how='left').to_sql('Salaries', FIN_ENGINE.connect(), if_exists='replace', index=False)

In [ ]:
customer_props = pd.read_sql('''select 
[Date], Person, Position, Customer, Hours, Ratio, Excluded_hours_proportionally, Not_excluded_hours_proportionally
from [DetailedBillability].[dbo].[Customers_Proportions]
where [Date] >= '2018-04-01'
and [Date] <= '2018-08-01'
''', DM_ENGINE.connect())

In [ ]:
customer_props.head()

In [ ]:
matched_employees = customer_props.groupby(['Person', 'Position']).apply(lambda x: get_matched_record(x, 'Person', 'Position', '')).reset_index()

In [ ]:
matched_employees.head()

In [ ]:
customer_props = matched_employees.merge(customer_props, how='outer')

In [ ]:
customer_props['Not_excluded_hours_proportionally'] = customer_props['Not_excluded_hours_proportionally'].fillna(0)
customer_props['Hours_With_Overhead'] = customer_props['Hours'] + customer_props['Not_excluded_hours_proportionally']
customer_props['Hours_With_Overhead'].hist()

In [ ]:
customer_props['Overhead_Ratio'] = customer_props['Not_excluded_hours_proportionally']/customer_props['Hours_With_Overhead']
customer_props['Overhead_Ratio'].hist()

In [ ]:
x = customer_props[(customer_props['Person'] == 'Jacek Krasnoborski') & (customer_props['Date'] == '2018-06-01')]

In [ ]:
real_customers = x[~x['Customer'].isin(['Objectivity_not_excl', 'Objectivity_excl'])]
holidays = x[x['Customer'] == 'Objectivity_excl']
obj = x[x['Customer'] == 'Objectivity_not_excl']
x['Nr_Of_Customers'] = real_customers.shape[0]
real_customers = x[~x['Customer'].isin(['Objectivity_not_excl', 'Objectivity_excl'])]

In [ ]:
obj

In [ ]:
def get_prop(x):
    real_customers = x[~x['Customer'].isin(['Objectivity_not_excl', 'Objectivity_excl'])]
    holidays = x[x['Customer'] == 'Objectivity_excl']
    obj = x[x['Customer'] == 'Objectivity_not_excl']
    x['Nr_Of_Customers'] = real_customers.shape[0]
    real_customers = x[~x['Customer'].isin(['Objectivity_not_excl', 'Objectivity_excl'])]
    
    if real_customers['Hours'].sum() > obj['Hours'].sum():
        real_customers['Hours_With_Overhead'] = real_customers['Hours'] + real_customers['Not_excluded_hours_proportionally']
    else:
        real_customers = x[~x['Customer'].isin(['Objectivity_excl'])]
        
        real_customers['Hours_With_Overhead'][real_customers['Customer'] != 'Objectivity_not_excl'] = \
        real_customers['Hours'] + obj['Hours'].sum() * real_customers['Hours'][real_customers['Customer'] != 'Objectivity_not_excl'] /real_customers['Hours'].sum()
        
        real_customers['Hours_With_Overhead'][real_customers['Customer'] == 'Objectivity_not_excl'] = \
            obj['Hours'].sum() * real_customers['Hours'][real_customers['Customer'] == 'Objectivity_not_excl'] /real_customers['Hours'].sum()
        
    real_customers['ProportionWithoutHolidays'] = real_customers['Hours_With_Overhead']/real_customers['Hours_With_Overhead'].sum()
    real_customers = pd.concat([real_customers, holidays])
    real_customers['Proportion'] = real_customers['Hours_With_Overhead']/real_customers['Hours_With_Overhead'].sum()
    
    real_customers = real_customers.drop(['level_2', 'Date', 'Person'], axis=1)
    return real_customers

In [ ]:
prop = customer_props.\
    groupby(['Date', 'Person']).apply(get_prop).reset_index()
prop[prop['Person'] == 'Jacek Kolonko']

In [ ]:
prop.to_sql('Customer_Proportions', FIN_ENGINE.connect(), if_exists='replace')

In [ ]:
prop.head()

In [ ]:
# TODO zwykłe proporcje gdzie obj też jest customerem